In [11]:
import torch
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from transformers import ViltProcessor, ViltModel
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef

In [12]:
file_path = "/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePUUM/1. Completed_Data/CanonBeetles.csv"

canon_df = pd.read_csv(file_path)

canon_df["ImageFilePath"] = canon_df["cropped_image_path"].apply(lambda x: f"/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePUUM/CANON/{x}")

df = canon_df[['ImageFilePath', 'Genus']]


In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "dandelin/vilt-b32-mlm"

model = ViltModel.from_pretrained(model_name).eval().to(device)

processor = ViltProcessor.from_pretrained(model_name)


In [14]:
def extract_features(image_path, taxa) :
    
    image = Image.open(image_path).convert("RGB")
    
    text = f"This is an image of a beetle, of genus : {taxa}"
    
    inputs = processor(images=image, text=text, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    features = outputs.last_hidden_state.mean(dim=1)
    
    return features.cpu().numpy().flatten()
    

In [15]:
X = np.vstack([extract_features(img, taxa) for img, taxa in zip(df["ImageFilePath"], df["Genus"])])

le = LabelEncoder()

y = le.fit_transform(df["Genus"])

df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df_indices, test_size=0.2, random_state=42)

test_df = df.loc[test_idx].copy()

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


In [16]:
seed = 99

models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "SVMLinear": SVC(kernel="linear"),
    "SVMPolynomial": SVC(kernel="poly", degree=4),
    "SVMRadialBasis": SVC(kernel="rbf", degree=4),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=5),
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=seed),    
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(256, 128), alpha=0.001, max_iter=250, random_state=seed)
}

predictions = {}

metrics = {}

for name, model in models.items():
    
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds
    
    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)
    
    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}
    # print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T


In [17]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,Genus,Pred_NaiveBayes,Pred_LogisticRegression,Pred_SVMLinear,Pred_SVMPolynomial,Pred_SVMRadialBasis,Pred_NearestNeighbor,Pred_RandomForest,Pred_MLP_Baseline
135,/fs/ess/PAS2136/Hawaii-2025/beetles_intake/Bee...,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax
844,/fs/ess/PAS2136/Hawaii-2025/beetles_intake/Bee...,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax


In [18]:
metrics_df

,Model,Accuracy,Precision,Recall,F1-Score,Balanced Acc,MCC
NaiveBayes,NaiveBayes,0.996923,0.99716,0.996923,0.996981,0.99842,0.993624
LogisticRegression,LogisticRegression,0.996923,0.996953,0.996923,0.996927,0.99842,0.993607
SVMLinear,SVMLinear,0.996923,0.996953,0.996923,0.996927,0.99842,0.993607
SVMPolynomial,SVMPolynomial,0.904615,0.915019,0.904615,0.895695,0.679793,0.799084
SVMRadialBasis,SVMRadialBasis,0.990769,0.991033,0.990769,0.990755,0.969063,0.980901
NearestNeighbor,NearestNeighbor,0.993846,0.993891,0.993846,0.993578,0.944444,0.987145
RandomForest,RandomForest,0.981538,0.982032,0.981538,0.979643,0.859531,0.961466
MLP_Baseline,MLP_Baseline,1.0,1.0,1.0,1.0,1.0,1.0


In [19]:
test_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/Beetle-PUUM/28.ViLT-linear-probing-genus.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/Beetle-PUUM/28.ViLT-linear-probing-genus-metrics.csv", index=False)